In [9]:
import monai
from monai.transforms import LoadImage
import os
import matplotlib.pyplot as plt

file_path = "data\\raw\\training_data\\BraTS-GLI-00005-100\\BraTS-GLI-00005-100-seg.nii.gz"
if os.path.exists(file_path):
    print("File path exists")
else:
    print("File path does not exist")



File path does not exist
